# Detección de objetos con DetectNet

La lista de objetos que se pueden detectar mediante las redes preentrenadas se encuentra en este [enlace](https://github.com/dusty-nv/jetson-inference/blob/master/data/networks/ssd_coco_labels.txt)

--------------

## Uso de los programas precompilados de la Jetson

Aquí se pueden encontrar los códigos de los programas precompilados:
 * [C++](https://github.com/dusty-nv/jetson-inference/blob/master/examples/detectnet/detectnet.cpp)
 * [Python](https://github.com/dusty-nv/jetson-inference/blob/master/python/examples/detectnet.py)

Ir a la carpeta con los programas precompilados con el siguiente comando

```
$ cd jetson-inference/build/aarch64/bin
```

A continuación, detectemos objetos en una imagen de ejemplo con el programap precompilado detectNet, tanto en C++ como en Python. Si está utilizando el contenedor Docker, es recomendable guardar la imagen de salida en el directorio images/test. Estas imágenes se podrán ver fácilmente desde su dispositivo host en el directorio jetson-inference/data/images/test.

```
# C++
$ ./detectnet images/peds_0.jpg images/test/peds_0.jpg     # (default network is SSD-Mobilenet-v2)

# Python
$ ./detectnet.py images/peds_0.jpg images/test/peds_0.jpg  # (default network is SSD-Mobilenet-v2)
```

Si queremos detectar varias imágenes

```
# C++
$ ./detectnet "images/peds_*.jpg" "images/test/peds_%i.jpg"     # (default network is SSD-Mobilenet-v2)

# Python
$ ./detectnet.py "images/peds_*.jpg" "images/test/peds_%i.jpg"  # (default network is SSD-Mobilenet-v2)
```

 > **nota**: cuando se usen asteriscos, hay que escribirlos siempre entre comillas ("*.jpg"). De lo contrario, el sistema operativo expandirá automáticamente la secuencia y modificará el orden de los argumentos en la línea de comandos, lo que puede resultar en que una de las imágenes de entrada sea sobrescrita por la salida.

Si queremos usar otras redes (por defecto se usa GoogleNet) hay que usar el flag ```--network```

```
# C++
$ ./detectnet --network=coco-airplane "images/airplane_*.jpg" "images/test/airplane_%i.jpg"

# Python
$ ./detectnet.py --network=coco-airplane "images/airplane_*.jpg" "images/test/airplane_%i.jpg"
```

Las redes que podemos usar para localización son:
|Network|argumento CLI|Network Type enum|Object classes|
|------|--------------|-----------------|--------------|
|SSD-Mobilenet-v1|`ssd-mobilenet-v1`|``SSD_MOBILENET_V1``|91 ([COCO classes](https://github.com/dusty-nv/jetson-inference/blob/master/data/networks/ssd_coco_labels.txt))|
|SSD-Mobilenet-v2|``ssd-mobilenet-v2``|``SSD_MOBILENET_V2``|91 ([COCO classes](https://github.com/dusty-nv/jetson-inference/blob/master/data/networks/ssd_coco_labels.txt))|
|SSD-Inception-v2|``ssd-inception-v2``|``SSD_INCEPTION_V2``|91 ([COCO classes](https://github.com/dusty-nv/jetson-inference/blob/master/data/networks/ssd_coco_labels.txt))|
|DetectNet-COCO-Dog|``coco-dog``|``COCO_DOG``|dogs|
|DetectNet-COCO-Bottle|``coco-bottle``|``COCO_BOTTLE``|bottles|
|DetectNet-COCO-Chair|``coco-chair``|``COCO_CHAIR``|chairs|
|DetectNet-COCO-Airplane|``coco-airplane``|``COCO_AIRPLANE``|airplanes|
|ped-100|``pednet``|``PEDNET``|pedestrians|
|multiped-500|``multiped``|``PEDNET_MULTI``|pedestrians, luggage|
|facenet-120|``facenet``|``FACENET``|faces|

Se puede elegir la información que sale en la imagen mediante el flag `--overlay`. Se puede elegir que aparezca una caja delimitadora (`box`), la etiqueta (`label`), la confianza (`conf`) o nada (`none`). Por defecto la configuración es `--overlay=box,labels,conf`.
```
# C++
$ ./detectnet --overlay=box,labels images/cat_0.jpg images/test/cat_0.jpg     # Only box and label

# Python
$ ./detectnet.py --overlay=box,labels images/cat_0.jpg images/test/cat_0.jpg  # Only box and label
```

Se puede elegir la transparencia de la caja mediante el flag `--alpha`, cuanto más pequeño es el valor más transparente es la caja. Por defecto `--alpha=120`

```
# C++
$ ./detectnet --alpha=50 images/cat_0.jpg images/test/cat_0.jpg      # Alpha 50
$ ./detectnet --alpha=200 images/cat_0.jpg images/test/cat_0.jpg     # Alpha 200

# Python
$ ./detectnet.py --alpha=50 images/cat_0.jpg images/test/cat_0.jpg   # Alpha 50
$ ./detectnet.py --alpha=200 images/cat_0.jpg images/test/cat_0.jpg  # Alpha 200
```

Se puede determinar el valor humbral para que la red considere un objeto mediante el flag `--threshold`. Por defecto `--threshold=0.5`

```
# C++
$ ./detectnet --threshold=0.2 images/peds_3.jpg images/test/peds_3.jpg     # threshold 0.2
$ ./detectnet --threshold=0.8 images/peds_3.jpg images/test/peds_3.jpg     # threshold 0.8

# Python
$ ./detectnet.py --threshold=0.2 images/peds_3.jpg images/test/peds_3.jpg  # threshold 0.2
$ ./detectnet.py --threshold=0.8 images/peds_3.jpg images/test/peds_3.jpg  # threshold 0.8
```

Si se quiere procesar un videdo solo hay que indicarlo en la entrada

Para ello ejecutamos el docker montando la carpeta del SDK de VisionWorks

```
$ docker/run.sh --volume /usr/share/visionworks/sources/data:/videos
```

Y ya lo podemos procesar

```
# C++
$ ./detectnet --threshold=0.35 /videos/pedestrians.mp4 images/test/pedestrians_ssd.mp4

# Python
$ ./detectnet.py --threshold=0.35 /videos/pedestrians.mp4 images/test/pedestrians_ssd.mp4
```

--------------

## Crear un programa de clasificación en Python

Como vamos a crear un programa, lo primero que tenemos que hacer es crear una carpeta en el Host donde guardaremos el programa

```
$ cd ~/
$ mkdir my-detection-python
$ cd my-detection-python
$ touch my-detection.py
$ chmod +x my-detection.py
```

Nos descargamos unas imágenes de osos para probar

```
$ wget https://github.com/dusty-nv/jetson-inference/raw/master/data/images/black_bear.jpg
$ wget https://github.com/dusty-nv/jetson-inference/raw/master/data/images/brown_bear.jpg
$ wget https://github.com/dusty-nv/jetson-inference/raw/master/data/images/polar_bear.jpg
```

A continuación lo que hay que hacer es lanzar el Docker con una carpeta del Host compartida, para que así cuando se cierre el Docker no se borre el programa, para ello lanzamos el Docker con el siguiente comando

```
$ docker/run.sh --volume ~/my-detection-python:/my-detection-python   # mounted inside the container to /my-detection-python
```

Una vez dentro del Docker ir a la carpeta con los siguientes comandos

```
$ cd ../
$ cd my-detection-python
```

Editar el archivo .py con el siguiente comando

```
$ nano my-detection.py
```

Para crear un programa como el precompilado escribimos el siguiente código

```Python
import jetson.inference
import jetson.utils

import argparse
import sys

# parse the command line
parser = argparse.ArgumentParser(description="Locate objects in a live camera stream using an object detection DNN.", 
                                 formatter_class=argparse.RawTextHelpFormatter, epilog=jetson.inference.detectNet.Usage() +
                                 jetson.utils.videoSource.Usage() + jetson.utils.videoOutput.Usage() + jetson.utils.logUsage())

parser.add_argument("input_URI", type=str, default="", nargs='?', help="URI of the input stream")
parser.add_argument("output_URI", type=str, default="", nargs='?', help="URI of the output stream")
parser.add_argument("--network", type=str, default="ssd-mobilenet-v2", help="pre-trained model to load (see below for options)")
parser.add_argument("--overlay", type=str, default="box,labels,conf", help="detection overlay flags (e.g. --overlay=box,labels,conf)\nvalid combinations are:  'box', 'labels', 'conf', 'none'")
parser.add_argument("--threshold", type=float, default=0.5, help="minimum detection threshold to use") 

is_headless = ["--headless"] if sys.argv[0].find('console.py') != -1 else [""]

try:
	opt = parser.parse_known_args()[0]
except:
	print("")
	parser.print_help()
	sys.exit(0)

# create video output object 
output = jetson.utils.videoOutput(opt.output_URI, argv=sys.argv+is_headless)
	
# load the object detection network
net = jetson.inference.detectNet(opt.network, sys.argv, opt.threshold)

# create video sources
input = jetson.utils.videoSource(opt.input_URI, argv=sys.argv)


# process frames until the user exits
while True:
	# capture the next image
	img = input.Capture()

	# detect objects in the image (with overlay)
	detections = net.Detect(img, overlay=opt.overlay)

	# print the detections
	print("detected {:d} objects in image".format(len(detections)))

	for detection in detections:
		print(f"\t{detection}")

		# find the object description
		class_description = net.GetClassDesc(detection.ClassID)
		print(f"\t clase {detection.ClassID}: {class_description}")

	# render the image
	output.Render(img)

	# update the title bar
	output.SetStatus("{:s} | Network {:.0f} FPS".format(opt.network, net.GetNetworkFPS()))

	# print out performance info
	net.PrintProfilerTimes()

	# exit on input/output EOS
	if not input.IsStreaming() or not output.IsStreaming():
		break

```

Ejecutar el programa con el siguiente comando

```
$ python3 my-detection.py /dev/video0
```

En este caso abrirá la webcam, se pueden introducir las mismas variables que con el programa precompilado

--------------

## Crear un programa ded clasificación en C++

Como vamos a crear un programa, lo primero que tenemos que hacer es crear una carpeta en el Host donde guardaremos el programa

```
$ cd ~/
$ mkdir my-detection-cpp
$ cd my-detection-cpp
$ touch my-detection.cpp
$ chmod +x my-detection.cpp
$ touch CMakeLists.txt
$ chmod +x CMakeLists.txt
```

Nos descargamos unas imágenes de osos para probar

```
$ wget https://github.com/dusty-nv/jetson-inference/raw/master/data/images/black_bear.jpg
$ wget https://github.com/dusty-nv/jetson-inference/raw/master/data/images/brown_bear.jpg
$ wget https://github.com/dusty-nv/jetson-inference/raw/master/data/images/polar_bear.jpg
```

A continuación lo que hay que hacer es lanzar el Docker con una carpeta del Host compartida, para que así cuando se cierre el Docker no se borre el programa, para ello lanzamos el Docker con el siguiente comando

```
$ docker/run.sh --volume ~/my-detection-cpp:/my-detection-cpp   # mounted inside the container to /my-detection-cpp
```

Una vez dentro del Docker ir a la carpeta con los siguientes comandos

```
$ cd ../
$ cd my-detection-cpp
```

Editar el archivo .py con el siguiente comando

```
$ nano my-detection.cpp
```

Para crear un programa como el precompilado escribimos el siguiente código

```C++
#include <jetson-utils/videoSource.h>
#include <jetson-utils/videoOutput.h>

#include <jetson-utils/cudaFont.h>
#include <jetson-inference/detectNet.h>

#include <signal.h>


#ifdef HEADLESS
	#define IS_HEADLESS() "headless"	// run without display
#else
	#define IS_HEADLESS() (const char*)NULL
#endif


bool signal_recieved = false;

void sig_handler(int signo)
{
	if( signo == SIGINT )
	{
		LogVerbose("received SIGINT\n");
		signal_recieved = true;
	}
}

int usage()
{
	printf("usage: detectnet [--help] [--network=NETWORK] [--threshold=THRESHOLD] ...\n");
	printf("                 input_URI [output_URI]\n\n");
	printf("Locate objects in a video/image stream using an object detection DNN.\n");
	printf("See below for additional arguments that may not be shown above.\n\n");
	printf("positional arguments:\n");
	printf("    input_URI       resource URI of input stream  (see videoSource below)\n");
	printf("    output_URI      resource URI of output stream (see videoOutput below)\n\n");

	printf("%s", detectNet::Usage());
	printf("%s", videoSource::Usage());
	printf("%s", videoOutput::Usage());
	printf("%s", Log::Usage());

	return 0;
}

int main( int argc, char** argv )
{
	/*
	 * parse command line
	 */
	commandLine cmdLine(argc, argv, IS_HEADLESS());

	if( cmdLine.GetFlag("help") )
		return usage();


	/*
	 * attach signal handler
	 */
	if( signal(SIGINT, sig_handler) == SIG_ERR )
		LogError("can't catch SIGINT\n");


	/*
	 * create input stream
	 */
	videoSource* input = videoSource::Create(cmdLine, ARG_POSITION(0));

	if( !input )
	{
		LogError("detectnet:  failed to create input stream\n");
		return 0;
	}


	/*
	 * create output stream
	 */
	videoOutput* output = videoOutput::Create(cmdLine, ARG_POSITION(1));
	
	if( !output )
		LogError("detectnet:  failed to create output stream\n");	
	

	/*
	 * create detection network
	 */
	detectNet* net = detectNet::Create(cmdLine);
	
	if( !net )
	{
		LogError("detectnet:  failed to load detectNet model\n");
		return 0;
	}

	// parse overlay flags
	const uint32_t overlayFlags = detectNet::OverlayFlagsFromStr(cmdLine.GetString("overlay", "box,labels,conf"));
	

	/*
	 * processing loop
	 */
	while( !signal_recieved )
	{
		// capture next image image
		uchar3* image = NULL;

		if( !input->Capture(&image, 1000) )
		{
			// check for EOS
			if( !input->IsStreaming() )
				break; 

			LogError("detectnet:  failed to capture video frame\n");
			continue;
		}

		// detect objects in the frame
		detectNet::Detection* detections = NULL;
	
		const int numDetections = net->Detect(image, input->GetWidth(), input->GetHeight(), &detections, overlayFlags);
		
		if( numDetections > 0 )
		{
			LogVerbose("%i objects detected\n", numDetections);
		
			for( int n=0; n < numDetections; n++ )
			{
				LogVerbose("detected obj %i  class #%u (%s)  confidence=%f\n", n, detections[n].ClassID, net->GetClassDesc(detections[n].ClassID), detections[n].Confidence);
				LogVerbose("bounding box %i  (%f, %f)  (%f, %f)  w=%f  h=%f\n", n, detections[n].Left, detections[n].Top, detections[n].Right, detections[n].Bottom, detections[n].Width(), detections[n].Height()); 
			}
		}	

		// render outputs
		if( output != NULL )
		{
			output->Render(image, input->GetWidth(), input->GetHeight());

			// update the status bar
			char str[256];
			sprintf(str, "TensorRT %i.%i.%i | %s | Network %.0f FPS", NV_TENSORRT_MAJOR, NV_TENSORRT_MINOR, NV_TENSORRT_PATCH, precisionTypeToStr(net->GetPrecision()), net->GetNetworkFPS());
			output->SetStatus(str);

			// check if the user quit
			if( !output->IsStreaming() )
				signal_recieved = true;
		}

		// print out timing info
		net->PrintProfilerTimes();
	}
	

	/*
	 * destroy resources
	 */
	LogVerbose("detectnet:  shutting down...\n");
	
	SAFE_DELETE(input);
	SAFE_DELETE(output);
	SAFE_DELETE(net);

	LogVerbose("detectnet:  shutdown complete.\n");
	return 0;
}

```

Editar el CMakeList.txt con lo siguiente

```
# require CMake 2.8 or greater
cmake_minimum_required(VERSION 2.8)

# declare my-detection project
project(my-detection)

# import jetson-inference and jetson-utils packages.
# note that if you didn't do "sudo make install"
# while building jetson-inference, this will error.
find_package(jetson-utils)
find_package(jetson-inference)

# CUDA and Qt4 are required
find_package(CUDA)
find_package(Qt4)

# setup Qt4 for build
include(${QT_USE_FILE})
add_definitions(${QT_DEFINITIONS})

# compile the my-detection program
cuda_add_executable(my-detection my-detection.cpp)

# link my-detection to jetson-inference library
target_link_libraries(my-detection jetson-inference)
```

Compilar el código con los siguientes comandos

```
$ cmake .
$ make
```

Ejecutar el programa con el siguiente comando

```
$ ./my-detection /dev/video0
```

En este caso abrirá la webcam, se pueden introducir las mismas variables que con el programa precompilado